In [3]:
import csv
import re
import spacy
from spacy import displacy

# pre-processing
def filter(senSet):
    #remove content between [ ]
    print("Pre-processing...")
    for index in range(len(senSet)):
        while senSet[index].find('[')>=0:
            i_start = senSet[index].find('[')
            i_end = senSet[index].find(']')
            s = senSet[index][i_start:i_end+2]
            senSet[index] = senSet[index].replace(s, "")

#load data
file = open("shortdataset.csv", "r")
#file = open("newdataset_formatted.csv", "r")
reader = csv.reader(file)

senSet = []
for item in reader:
    #format sentences in item as string
    fullP = "".join(item)
    splitP = fullP.split(";", 3);
    splitS = splitP[3][1:len(splitP[3])].split(".");
    #print(splitS)
    for sen in splitS:
        senSet.append(sen)#store the sentence into an array

file.close()
print("Total sentences: " + str(len(senSet)))

#pre-processing
filter(senSet)

'''for sentence in senSet:
    print(senSet)'''

#parse sentence
nlp = spacy.load('en_core_web_sm')

#from spacy.lang.en.stop_words import STOP_WORDS
#print(STOP_WORDS)
for index in range(len(senSet)):
    #index = 1
    print(str(index) + ":" + senSet[index])
    doc = nlp(unicode(senSet[index]))#For mac: doc = nlp(str(senSet[index]))
    #print the result
    sub = ""
    pred = ""
    obj = ""
    for token in doc:
        #filter stop words
        #print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
        #      token.shape_, token.is_alpha, token.is_stop)
        #print(token.text,  token.is_stop)
        if re.match('nsubj', token.dep_):   
            subj = token.text
        if re.match('ROOT', token.dep_): 
            pred = token.lemma_
            pred_orig = token.text
        if re.match('dobj', token.dep_): 
            obj = token.lemma_

    for chunk in doc.noun_chunks:
        #print(chunk.text, chunk.root.text, chunk.root.dep_, chunk.root.head.text)
        if chunk.root.head.text == pred_orig and chunk.root.dep_ == "nsubj":
            subj = chunk.text
        if chunk.root.head.text == pred_orig and re.match('dobj|attr', chunk.root.dep_):
            obj = chunk.text

    print(subj + ' - ' + pred + ' - ' + obj + '\n')

    ## visualize the semantic tree
    #options = {'compact': True, 'color': 'blue'}
    #displacy.serve(doc, style='dep', options=options)
    #displacy.serve(doc, style='dep')'''

Total sentences: 46
Pre-processing...
0:Sure um start off - I'm a Purdue grad
I - be - a Purdue grad

1:  Um my title is director of landscape architecture so um  for the ah staff of people
my title - be - director

2:  Um we manage all the construction and innovation projects for the university across the state
we - manage - all the construction and innovation projects

3:   Some projects we handle in-house we design these others we're managing and consulting on your architecture 
we - design - these others

4:  Um and we've done quite a bit in here in the last five years or so including 
we - do - 

5:  Um typically this camp will contact me and say 'We have this project we want to do
this camp - contact - me

6:  How do we go about getting it accomplished?'  Um that's when I decide our staff will handle this do the -
staff - be - the

7:- design in-house or if it's big enough that we need some outside help we call up XYZ consultants and bring them in to work on it
we - design - -PRO